In [26]:
import platform as python_platform

class Platform:
    WINDOWS = "Windows"
    MACOS = "Darwin"

platform = python_platform.system()

In [27]:
windows_root_directory = "D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling"
macos_root_directory = "???"

root_directory = windows_root_directory if platform == Platform.WINDOWS else macos_root_directory

In [28]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

import sys
sys.path.append(root_directory)

import os
from os.path import join, getsize
import threading
import time
import subprocess
import signal

import re

import file_utils as fu

In [29]:
json_dir_path = fu.relative_to_absolute_path(
    "GetAllSongHTML/song_list_link_by_artist/json",
    root_path=root_directory
)

print(json_dir_path)

D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json


In [30]:
song_links_by_alphabet = fu.get_file_path_list_in_dir(json_dir_path)

song_links_by_alphabet = {
    path.split("/")[-1].split(".")[0].lower() : path
    for path in song_links_by_alphabet
}

print(song_links_by_alphabet)

{'a': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/a.json', 'b': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/b.json', 'c': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/c.json', 'd': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/d.json', 'e': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/e.json', 'f': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/f.json', 'g': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/g.json', 'h': 'D:/Vector A/0. KHTN/Nam 4/HKII/Thesis/Brainstorming/DataCrawling/GetAllSongHTML/song_list_link_by_artist/json/h.json',

In [31]:
def song_link_to_relative_html_file_path(link):
    artist_letter = link.split("/")[-2][0]
    artist = link.split("/")[-2]
    song = link.split("/")[-1]

    return f"{artist_letter}/{artist}/{song}.html"


print(
    song_link_to_relative_html_file_path(
        fu.read_data_from_json_file(
            song_links_by_alphabet["a"])[0]["link"]
        )
    )

a/a-boogie-wit-da-hoodie/drowning.html


In [32]:
webdriver_pool = []

In [33]:
def scroll_to_bottom(webdriver_pool_index, scroll_pause_time = 0.5):
    # Get the screen height of the web
    screen_height = webdriver_pool[webdriver_pool_index]["browser"].execute_script("return window.screen.height;") 
    i = 1

    max_scroll_times = 25

    while True:
        print(f"Scrolled: {i} time(s)")

        # Scroll one screen height each time
        try:
            webdriver_pool[webdriver_pool_index]["browser"].execute_script(f"window.scrollTo(0, {screen_height}*{i});")
        except:
            print(f"Error while scrolling")
            raise Exception("Error while scrolling")
        
        time.sleep(scroll_pause_time)
        
        # Cập nhật scroll_height sau mỗi lần scroll, vì scroll_height có thể thay đổi sau khi scroll trang
        scroll_height = webdriver_pool[webdriver_pool_index]["browser"].execute_script("return document.body.scrollHeight;")

        i += 1
        if (screen_height) * i > scroll_height:
            break

        if i > max_scroll_times:
            print(f"Max scroll times reached")
            raise Exception("Max scroll times reached")

In [34]:
def scroll_and_get_html_content(url, webdriver_pool_index):
    try:
        webdriver_pool[webdriver_pool_index]["browser"].get(url)
    except:
        print(f"Failed to get {url}")
        raise Exception(f"Failed to get {url}")
    
    scroll_to_bottom(webdriver_pool_index)
    
    contents = webdriver_pool[webdriver_pool_index]["browser"].page_source

    return contents

In [35]:
def get_song_raw_html(url, webdriver_pool_index):
    print(f"{url}: processing...")
    
    page_source = scroll_and_get_html_content(url, webdriver_pool_index)

    path_to_save_html = fu.relative_to_absolute_path(
        song_link_to_relative_html_file_path(url),
        root_path=fu.relative_to_absolute_path(
            "GetAllSongHTML/song_list/raw_html",
            root_path=root_directory
        )
    )
    
    os.makedirs(os.path.dirname(path_to_save_html), exist_ok=True)

    fu.write_data_to_html_file(page_source, path_to_save_html)

    print(f"{url}: done")

In [36]:
def open_edge_in_remote_debugging_mode(port, platform, browser_instance_data_dir):
    print(f"Opening Edge in remote debugging mode on port {port}...")

    macos_command = f'''open -na "Microsoft Edge.app" --args
                        --remote-debugging-port={port}
                        --user-data-dir="{browser_instance_data_dir}"'''

    windows_command = f'''start msedge.exe 
                        --remote-debugging-port={port} 
                        --user-data-dir="{browser_instance_data_dir}'''

    command = macos_command if platform == Platform.MACOS else windows_command
    
    os.system(command.replace("\n", " "))

In [37]:
def init_selenium_in_remote_debugging_mode(port):
    options = webdriver.EdgeOptions()
    options.add_experimental_option("debuggerAddress", f"localhost:{port}")

    # browser.maximize_window()
    
    webdriver_pool.append(
        {
            "port": port,
            "browser": webdriver.Edge(options=options),
            "browser_status": BrowserStatus.AVAILABLE
        }
    )

In [38]:
def windows_get_PID_of_process_running_on_port(port):
    command = f"netstat -a -n -o | findstr :{port}"

    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    #   TCP    127.0.0.1:40000        0.0.0.0:0              LISTENING       6920
    regex_pattern = r"\s+TCP\s+127.0.0.1:\d+\s+\d+.\d+.\d+.\d+:\d+\s+LISTENING\s+(.+)"

    match = re.search(regex_pattern, result.stdout)

    pid = None

    if match:
        pid = match.group(1).replace(" ", "")
    else:
        pid = None

    return pid

In [39]:
def kill_process_running_on_port(port, platform=Platform.WINDOWS):
    print(f"Killing process running on port {port}...")

    try:
        if platform == Platform.MACOS:
            result = os.system(f"lsof -ti tcp:{port} | xargs kill -9")
        else:
            pid = windows_get_PID_of_process_running_on_port(port)

            if pid != None:
                result = subprocess.run(
                    f"taskkill /PID {pid} /F", 
                    shell=True, 
                    capture_output=True, 
                    text=True
                ).stdout
            else:
                result = f"No process is running on port {port}"
    except:
        print(f"Error when killing process running on port {port}")

In [40]:
class BrowserStatus:
    AVAILABLE = "available"
    BUSY = "busy"

In [41]:
def start_webdriver(port, platform):
    # Start webdriver if port is not in the pool
    if len(webdriver_pool) == 0 or port not in [webdriver["port"] for webdriver in webdriver_pool]:
        kill_process_running_on_port(port, platform)

        open_edge_in_remote_debugging_mode(
            port,
            platform,
            fu.relative_to_absolute_path(
                f"GetAllSongHTML/song_list/browser_instance_data/{port}",
                root_path=root_directory
            )
        )

        init_selenium_in_remote_debugging_mode(port)
    else:
        print(f"Webdriver on port {port} is already started")

In [42]:

def init_webdriver_pool(n_webdriver=5):

    webdriver_pool.clear()

    if platform == Platform.WINDOWS:
        os.system("taskkill /F /IM msedgedriver.exe")
        os.system("taskkill /F /IM msedge.exe")

    for i in range(n_webdriver):
        port = 40000 + i

        kill_process_running_on_port(port, platform)
        
        start_webdriver_thread = threading.Thread(
            target=start_webdriver, 
            args=(port, platform, )
        )
        
        start_webdriver_thread.start()

        start_webdriver_thread.join(15)

        # Check if the webdriver is started successfully
        if len(webdriver_pool) != i + 1:
            print(f"Failed to start webdriver on port {port}. Retrying...")
            i -= 1
        else:
            print(f"Webdriver on port {port} started successfully")           

In [43]:
def restart_webdriver_pool():
    n_webdriver_item = len(webdriver_pool)

    for webdriver_instance in webdriver_pool:
        kill_process_running_on_port(webdriver_instance["port"], platform)

    init_webdriver_pool(n_webdriver_item)

In [44]:
def change_port_status(port, status):
    for i in range(len(webdriver_pool)):
        if webdriver_pool[i]["port"] == port:
            webdriver_pool[i]["browser_status"] = status

In [45]:
def get_raw_html_with_webdriver_from_pool(url, webdriver_pool_index):
    if (webdriver_pool[webdriver_pool_index]["browser_status"] == BrowserStatus.AVAILABLE):
        try:
            change_port_status(webdriver_pool[webdriver_pool_index]["port"], BrowserStatus.BUSY)
            get_song_raw_html(url, webdriver_pool_index)
            change_port_status(webdriver_pool[webdriver_pool_index]["port"], BrowserStatus.AVAILABLE)
        except:
            restart_webdriver_pool()

In [46]:
def get_song_raw_html_by_alphabet(letter, song_per_batch=5):
    # Lấy danh sách link của letter
    song_links = fu.read_data_from_json_file(song_links_by_alphabet[letter])

    # Xóa các song đã được crawl
    song_links = [song for song in song_links
        if not os.path.exists(
            fu.relative_to_absolute_path(
                song_link_to_relative_html_file_path(song["link"]),
                root_path=fu.relative_to_absolute_path(
                    "GetAllSongHTML/song_list/raw_html",
                    root_path=root_directory
                )
            )
        )
    ]

    n_song_processed = 0
    thread_timeout = 60

    while(len(song_links) > 0):
        current_song_per_batch = song_per_batch if song_per_batch <= len(song_links) else len(song_links)
        song_batch = [song_links.pop(0) for _ in range(current_song_per_batch)]

        threadBatch = []

        for webdriver_pool_index in range(len(webdriver_pool)):
            if webdriver_pool[webdriver_pool_index]["browser_status"] == BrowserStatus.AVAILABLE:
                threadBatch.insert(
                    0,
                    threading.Thread(
                        target=get_raw_html_with_webdriver_from_pool,
                        args=(song_batch.pop(0)["link"], webdriver_pool_index, )
                    )
                )

                threadBatch[0].start()
            elif webdriver_pool_index == len(webdriver_pool) - 1:
                print(f"All webdrivers are busy")
                break
        
        for i in range(len(threadBatch)):
            threadBatch[i].join(thread_timeout)


        anti_rate_limiting_waited_time = 0
        total_time_to_wait = 30

        for i in range(total_time_to_wait):
            time.sleep(1)
            anti_rate_limiting_waited_time += 1
            print(f"Waited {anti_rate_limiting_waited_time} of {total_time_to_wait} seconds")
        
        threadBatch.clear()

        n_song_processed += current_song_per_batch
        print(f"{n_song_processed} songs processed")

In [47]:
song_per_batch = 5
init_webdriver_pool(n_webdriver=song_per_batch)

Killing process running on port 40000...
Killing process running on port 40000...
Opening Edge in remote debugging mode on port 40000...
Webdriver on port 40000 started successfully
Killing process running on port 40001...
Killing process running on port 40001...
Opening Edge in remote debugging mode on port 40001...
Webdriver on port 40001 started successfully
Killing process running on port 40002...
Killing process running on port 40002...
Opening Edge in remote debugging mode on port 40002...
Webdriver on port 40002 started successfully
Killing process running on port 40003...
Killing process running on port 40003...
Opening Edge in remote debugging mode on port 40003...
Webdriver on port 40003 started successfully
Killing process running on port 40004...
Killing process running on port 40004...
Opening Edge in remote debugging mode on port 40004...
Webdriver on port 40004 started successfully


In [ ]:
get_song_raw_html_by_alphabet('a', song_per_batch)

https://www.hooktheory.com/theorytab/view/akira-yamaoka/promise: processing...
https://www.hooktheory.com/theorytab/view/alstroemeria-records/bad-apple: processing...
https://www.hooktheory.com/theorytab/view/alstroemeria-records/lovelight---bad-apple: processing...
https://www.hooktheory.com/theorytab/view/amber-island/big-blue-bubble: processing...
https://www.hooktheory.com/theorytab/view/amber-run/i-found: processing...
Scrolled: 1 time(s)
Scrolled: 1 time(s)
Scrolled: 1 time(s)
Scrolled: 2 time(s)
Scrolled: 1 time(s)
Scrolled: 1 time(s)
Scrolled: 3 time(s)
Scrolled: 2 time(s)
Scrolled: 3 time(s)
Scrolled: 2 time(s)
Scrolled: 4 time(s)
Scrolled: 4 time(s)
Scrolled: 2 time(s)
Scrolled: 5 time(s)
https://www.hooktheory.com/theorytab/view/amber-island/big-blue-bubble: done
https://www.hooktheory.com/theorytab/view/amber-run/i-found: done
Scrolled: 3 time(s)
Scrolled: 3 time(s)
Scrolled: 4 time(s)
Scrolled: 4 time(s)
Scrolled: 5 time(s)
Scrolled: 5 time(s)
Scrolled: 6 time(s)
Scrolled:

Exception in thread Thread-772 (start_webdriver):
Traceback (most recent call last):
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Vector A\AppData\Local\Temp\ipykernel_23920\2602687824.py", line 15, in start_webdriver
  File "C:\Users\Vector A\AppData\Local\Temp\ipykernel_23920\2828608030.py", line 10, in init_selenium_in_remote_debugging_mode
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\edge\webdriver.py", line 45, in __init__
    super().__init__(
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 50, in __init__
    self.service.start()
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python31

Failed to start webdriver on port 40001. Retrying...
Killing process running on port 40002...
Killing process running on port 40002...
Opening Edge in remote debugging mode on port 40002...


Exception in thread Thread-763 (start_webdriver):
Traceback (most recent call last):
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 449, in _make_request
    six.raise_from(e, None)
  File "<string>", line 3, in raise_from
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\site-packages\urllib3\connectionpool.py", line 444, in _make_request
    httplib_response = conn.getresponse()
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 1374, in getresponse
    response.begin()
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 318, in begin
    version, status, reason = self._read_status()
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Pyth

Failed to start webdriver on port 40001. Retrying...
Killing process running on port 40002...
Killing process running on port 40002...
Opening Edge in remote debugging mode on port 40002...
Failed to start webdriver on port 40002. Retrying...
Killing process running on port 40003...
Failed to start webdriver on port 40001. Retrying...
Killing process running on port 40002...
Webdriver on port 40002 is already started
Failed to start webdriver on port 40002. Retrying...
Killing process running on port 40003...
Killing process running on port 40003...
Opening Edge in remote debugging mode on port 40003...
Failed to start webdriver on port 40003. Retrying...
Killing process running on port 40004...
Killing process running on port 40004...
Opening Edge in remote debugging mode on port 40004...
Webdriver on port 40003 is already started
Failed to start webdriver on port 40003. Retrying...
Killing process running on port 40004...
Killing process running on port 40004...
Opening Edge in remot

...

Killing process running on port 40000...
Webdriver on port 40000 is already started
Webdriver on port 40000 started successfully
Killing process running on port 40001...
Killing process running on port 40001...
Opening Edge in remote debugging mode on port 40001...
Failed to start webdriver on port 40004. Retrying...
Killing process running on port 40005...
Killing process running on port 40005...
Opening Edge in remote debugging mode on port 40005...
Webdriver on port 40001 started successfully
Killing process running on port 40002...
Killing process running on port 40002...
Opening Edge in remote debugging mode on port 40002...
Webdriver on port 40001 started successfully
Killing process running on port 40002...
Killing process running on port 40002...
Opening Edge in remote debugging mode on port 40002...
Failed to start webdriver on port 40005. Retrying...
Killing process running on port 40006...
Killing process running on port 40006...
Opening Edge in remote debugging mode on port

Exception in thread Thread-1468 (start_webdriver):
Traceback (most recent call last):
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Vector A\AppData\Local\Temp\ipykernel_23920\2602687824.py", line 15, in start_webdriver
  File "C:\Users\Vector A\AppData\Local\Temp\ipykernel_23920\2828608030.py", line 10, in init_selenium_in_remote_debugging_mode
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\edge\webdriver.py", line 45, in __init__
    super().__init__(
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 50, in __init__
    self.service.start()
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python3

Webdriver on port 40004 is already started
Failed to start webdriver on port 40004. Retrying...
Killing process running on port 40005...
Webdriver on port 40005 is already started
Failed to start webdriver on port 40005. Retrying...


Exception in thread Thread-1504 (start_webdriver):
Traceback (most recent call last):
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "C:\Users\Vector A\AppData\Local\Temp\ipykernel_23920\2602687824.py", line 15, in start_webdriver
  File "C:\Users\Vector A\AppData\Local\Temp\ipykernel_23920\2828608030.py", line 10, in init_selenium_in_remote_debugging_mode
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\edge\webdriver.py", line 45, in __init__
    super().__init__(
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python310\lib\site-packages\selenium\webdriver\chromium\webdriver.py", line 50, in __init__
    self.service.start()
  File "c:\Users\Vector A\AppData\Local\Programs\Python\Python3

Failed to start webdriver on port 40008. Retrying...
Killing process running on port 40009...
Killing process running on port 40009...
Opening Edge in remote debugging mode on port 40009...
Failed to start webdriver on port 40008. Retrying...
Killing process running on port 40009...
Failed to start webdriver on port 40008. Retrying...
Killing process running on port 40009...
Killing process running on port 40009...
Opening Edge in remote debugging mode on port 40009...
Killing process running on port 40009...
Opening Edge in remote debugging mode on port 40009...
Failed to start webdriver on port 40009. Retrying...
Killing process running on port 40010...
Killing process running on port 40010...
Opening Edge in remote debugging mode on port 40010...
Failed to start webdriver on port 40009. Retrying...
Killing process running on port 40010...
Failed to start webdriver on port 40009. Retrying...
Killing process running on port 40010...
Killing process running on port 40010...
Killing pro

In [ ]:
# Kill all Edge instances
for webdriver in webdriver_pool:
    kill_process_running_on_port(webdriver["port"], platform)

In [ ]:
import platform

# Lấy thông tin về hệ điều hành
operating_system = platform.system()

print(operating_system)


In [49]:
# For debugging
import os
import platform as python_platform

if python_platform.system() == "Windows":
    os.system("taskkill /F /IM msedge.exe")